## calculate change ratios and combine common data

### feature(주요 common data) 항목 변화율 계산하고 합치기-- 하루전과 이틀전의 데이터와 비교

In [1]:
import datetime, time
# from datetime import date

import pandas as pd
import numpy as np

In [2]:
# 개장일(date)과 이전 개장일(date_p1), 이전이전 개장일(date_p2)을 dict로 구성
base_data_directory = './data/base_data/stock_market_holydays/'
OPENING_DAYS_KOR = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
OPENING_DAYS_USA = pd.read_pickle(base_data_directory+'opening_days_usa.pkl') # 미국 개장일 데이터 

In [3]:
'''
# 한국 시장 예측 대상이 되는 개장일과 예측데이터를 계산한 미국 개장일과 예측 데이터 일치 시키기.
MATCHING_DAYS = pd.read_csv(base_data_directory+'find_delete_date.csv', encoding='cp949') # 미국 개장일 데이터 

MATCHING_DAYS['date_kor'] =  pd.to_datetime(MATCHING_DAYS['date_kor'],format='%Y-%m-%d').dt.date
MATCHING_DAYS['date_all'] =  pd.to_datetime(MATCHING_DAYS['date_all'],format='%Y-%m-%d').dt.date

for x in range(len(MATCHING_DAYS['replace_date'])):
    try:
        MATCHING_DAYS['replace_date'][x] =  datetime.datetime.strptime(MATCHING_DAYS['replace_date'][x], '%Y-%m-%d').date()
    except:
        pass
    
for x in range(len(MATCHING_DAYS['date_usa'])):
    try:
        MATCHING_DAYS['date_usa'][x] =  datetime.datetime.strptime(MATCHING_DAYS['date_usa'][x], '%Y-%m-%d').date()
    except:
        pass
    
MATCHING_DAYS.tail()

MATCHING_DAYS.to_pickle(base_data_directory + 'matching_days.pkl')
MATCHING_DAYS.to_csv(base_data_directory + 'matching_days.csv')
'''

"\n# 한국 시장 예측 대상이 되는 개장일과 예측데이터를 계산한 미국 개장일과 예측 데이터 일치 시키기.\nMATCHING_DAYS = pd.read_csv(base_data_directory+'find_delete_date.csv', encoding='cp949') # 미국 개장일 데이터 \n\nMATCHING_DAYS['date_kor'] =  pd.to_datetime(MATCHING_DAYS['date_kor'],format='%Y-%m-%d').dt.date\nMATCHING_DAYS['date_all'] =  pd.to_datetime(MATCHING_DAYS['date_all'],format='%Y-%m-%d').dt.date\n\nfor x in range(len(MATCHING_DAYS['replace_date'])):\n    try:\n        MATCHING_DAYS['replace_date'][x] =  datetime.datetime.strptime(MATCHING_DAYS['replace_date'][x], '%Y-%m-%d').date()\n    except:\n        pass\n    \nfor x in range(len(MATCHING_DAYS['date_usa'])):\n    try:\n        MATCHING_DAYS['date_usa'][x] =  datetime.datetime.strptime(MATCHING_DAYS['date_usa'][x], '%Y-%m-%d').date()\n    except:\n        pass\n    \nMATCHING_DAYS.tail()\n\nMATCHING_DAYS.to_pickle(base_data_directory + 'matching_days.pkl')\nMATCHING_DAYS.to_csv(base_data_directory + 'matching_days.csv')\n"

In [4]:
MATCHING_DAYS = pd.read_pickle(base_data_directory + 'matching_days.pkl')
usa_delete = MATCHING_DAYS[MATCHING_DAYS['replace_date'] == 'delete'][['date_usa', 'replace_date']] # usa date delete
usa_replace = MATCHING_DAYS[MATCHING_DAYS['date_usa'].isna()][['date_kor', 'replace_date']] #use date replace
# usa_replace.columns = ['date_usa', 'replace_date']

In [5]:
MATCHING_DAYS

,date_all,date_kor,date_usa,replace_date
0,2020-01-20,2020-01-20,NaN,2020-01-21
1,2020-01-21,2020-01-21,2020-01-21,delete
2,2020-02-17,2020-02-17,NaN,2020-02-18
3,2020-02-18,2020-02-18,2020-02-18,delete
4,2020-04-10,2020-04-10,NaN,2020-04-13
...,...,...,...,...
62,2023-07-05,2023-07-05,2023-07-05,delete
63,2023-09-04,2023-09-04,NaN,2023-09-05
64,2023-09-05,2023-09-05,2023-09-05,delete
65,2023-11-23,2023-11-23,NaN,2023-11-24


In [6]:
# 매일 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들(cpi, bok_rate, fed_rate 등)은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
              'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
              'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
              'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]
### ixf, qnet 파일 추가후 훈련시켜야 함.

common_pkl_usa = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
                  "ixic_future.pkl", "nas.pkl", "snp_future.pkl", "sox.pkl", "spx.pkl", 
                  "us_10yr_bond.pkl", "us_2yr_bond.pkl", "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
                  'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
                  'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
                  'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]
common_pkl_kor = [ "kor_10yr_bond.pkl", "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", 
             ]
# common_pkl = [ "dji.pkl" ]

In [7]:
# dict로 만들어 pkl내에 있는 df의 column 이름과 매칭시키기 위하여 구성
common_col_name = {'dji.pkl':'dji', 'dji_future.pkl':'dji_f', 'dxy_future.pkl':'dxy', 
                   'ixic_future.pkl':'ixic_f', 'kor_10yr_bond.pkl':'bond_kor_10',
                   'kor_2yr_bond.pkl':'bond_kor_2', 'kosdaq.pkl':'kosdaq', 'kospi.pkl':'kospi',
                   'krw_rate.pkl':'krw', 'nas.pkl':'ixic', 'snp_future.pkl':'spx_f',
                   'sox.pkl':'sox', 'spx.pkl':'spx', 'us_10yr_bond.pkl':'bond_usa_10',
                   'us_2yr_bond.pkl':'bond_usa_2', 'us_3mon_bond.pkl':'bond_usa_3m',
                   'vix.pkl':'vix', 'wti_future.pkl':'wti',
                   'spsy.pkl':'spsy', 'spny.pkl':'spny', 'spxhc.pkl':'spxhc', 'splrcd.pkl':'splrcd', 
                   'splrci.pkl':'splrci', 'splrcu.pkl':'splrcu', 'splrcs.pkl':'splrcs', 
                   'splrct.pkl':'splrct', 'splrcl.pkl':'splrcl', 'splrcm.pkl':'splrcm', 
                   'ixbk.pkl':'ixbk', 'ixfn.pkl':'ixfn', 'ixid.pkl':'ixid', 'ixis.pkl':'ixis', 
                   'ixk.pkl':'ixk', 'ixtr.pkl':'ixtr', 'ixut.pkl':'ixut', 'nbi.pkl':'nbi', 'bkx.pkl':'bkx'
                   }

In [8]:
directory_for_predict = './data/data_for_ml/predict/'
pkl_common_directory = './data/common_pkl/'
pkl_directory = './data/common_pkl/'

In [9]:
# 주요 지표를 한개의 df로 구성 (휴장일과 개장일이 틀리기 때문에 한국, 미국 각각 구성해야 함)
# _common : 공통으로 사용되는 지표를 의미
def get_common_base(opening_days_common, pkl_list):
    df_base = pd.DataFrame(pd.Series(opening_days_common, name='date')).set_index('date')
    date_compare = pd.DataFrame(pd.Series(opening_days_common, name='date'))
    date_compare['date_p'] = date_compare['date'].shift(1)
    
    date_compare.columns = ['date_c', 'date_p'] #거래일과 바로 이전 거래일
    c_p_dict = date_compare.set_index('date_c').to_dict()['date_p'] # date_c로 date_p 찾기
    p_c_dict = date_compare.set_index('date_p').to_dict()['date_c'] # date_p로 date_c 찾기
    
    for index_name in pkl_list:
        key_name = index_name[:-4]
        col_name = common_col_name[index_name]
        df_temp = pd.read_pickle(pkl_common_directory + index_name)
        
#         df_temp['date'] = df_temp['date'].dt.date # date type으로 변환
        df_temp['temp'] = df_temp[col_name].shift(2)  # 거래일과 이전(-1) 이전(-1) 거래일 데이터와 비교하기 위하여 -2 shift
        
        df_temp[f'{col_name}_cr_2'] = (df_temp[col_name] - df_temp['temp'])/df_temp['temp']*100 # 이전 거래일 데이터는 "_cr"
        df_temp[f'{col_name}_cr_2'] = df_temp[f'{col_name}_cr_2'].apply(lambda x: float(f'{x:.5f}')) # 5 소수점까지
        
        df_new = df_temp[df_temp['date'].isin(date_compare['date_p'])] # 이전 거래일과 비교날짜가 있는 row만 선택
        # 예: 2023-04-10('date_c'), 2023-04-07('date_p') 일 경우, 2023년 4월 7일 데이터는 2023년 4월 10일 row에 작성해야 함.
        # 위 예와 같이 하기 위해 'date_p'에 있는 date로만 df재구성(df_new)

        add_c= df_new['date'].apply(lambda x : p_c_dict[x])  # 하루전 날짜와 매칭한 column 생성 (add_c)
        # 예: 2023년 4월 7일 데이터는 2023년 4월 10일 row에 기입
        df_new.insert(len(df_new.columns)-1, 'date_c', value=add_c)  #SettingWithCopyWarning: error 회피
        df_new.set_index('date_c', inplace=True)

        df_base = df_base.merge(df_new.iloc[:, [-3, -1]], \
                                how='left', left_index=True, right_index=True)
    return df_base

In [10]:
# ***미국 주식 분석용 시작 일자, 마지막 일자  지정 ***********
# ** 한국 공통(kospi 등) 자료는 한국 개장일에 맞춰 분석해야 함.
pkl_name= 'dji.pkl'
df_temp = pd.read_pickle(pkl_directory + pkl_name)

dji = df_temp['dji'].iloc[-1]
opening_days_common = list(df_temp['date'])

# 오늘 날짜(분석날짜) 추가 필요(미국, 한국 포함)

if dji == None :
    opening_days_common = list(df_temp['date'][:-1])
    
# 마지막 기록일자 index 찾고 다음 개장일 추가하기 (예측을 원하는 날)
l_date = opening_days_common[-1]
lp1_index = list(OPENING_DAYS_USA).index(l_date) + 1 # 현재 개장일 이후에 오는 개장일 날짜 index (+1 index)
opening_days_common.append(list(OPENING_DAYS_USA)[lp1_index])

In [11]:
# 미국 common data df
df_usa = get_common_base(opening_days_common, common_pkl_usa)

In [12]:
df_usa.tail()

,dji_cr,dji_cr_2,dji_f_cr,dji_f_cr_2,dxy_cr,dxy_cr_2,ixic_f_cr,ixic_f_cr_2,ixic_cr,ixic_cr_2,...,ixk_cr,ixk_cr_2,ixtr_cr,ixtr_cr_2,ixut_cr,ixut_cr_2,nbi_cr,nbi_cr_2,bkx_cr,bkx_cr_2
date,,,,,,,,,,,,,,,,,,,,,
2024-03-14,0.10%,0.70586,0.10,0.73151,-0.16,-0.07777,-0.81,0.61621,-0.54%,0.98943,...,-0.81%,1.44914,-0.02%,0.17420,0.16%,-0.44686,0.26%,-0.19686,0.54%,0.46301
2024-03-15,-0.35%,-0.25593,-0.44,-0.33688,0.56,0.38850,-0.31,-1.11218,-0.30%,-0.84294,...,0.07%,-0.74541,-1.41%,-1.43434,-0.90%,-0.74762,-2.02%,-1.76188,-1.78%,-1.24912
2024-03-18,-0.49%,-0.84150,-0.49,-0.93115,0.07,0.62263,-1.17,-1.47175,-0.96%,-1.26470,...,-1.27%,-1.20060,0.38%,-1.04183,-0.86%,-1.75082,0.21%,-1.80988,0.02%,-1.76334
2024-03-19,0.20%,-0.29618,0.18,-0.31514,0.00,0.06772,0.96,-0.22029,0.82%,-0.15550,...,1.03%,-0.24581,0.07%,0.44877,0.32%,-0.54305,-0.65%,-0.43660,0.49%,0.51005
2024-03-20,0.83%,1.02285,0.81,0.98588,0.42,0.42541,0.21,1.16979,0.39%,1.21216,...,0.40%,1.44197,0.19%,0.25652,-0.11%,0.20698,0.81%,0.15416,0.20%,0.69352


In [13]:
# df_kor와 df_usa predict날짜 일치시키기 위하여 df_date_compare이용\
# 예. 예측일 기준 2023년 6월 19일의 경우, 미국시장 6월 20일 데이터를 사용하여야 함.
# 6월 20일 경우에는 
# 아래 로직을 새롭게 만들어서 사용해야 함.

In [14]:
# 예측 날짜 replace
as_list = df_usa.index.tolist()
for i in range(len(usa_replace)):
    usa_date =  usa_replace['date_kor'].iloc[i]
#     print("usadate ", usa_date, "  최초 기록날짜 ", as_list[0])
#     print("usadate ", usa_date, "  최초 기록날짜 ", as_list[-1])
    if usa_date < as_list[0]: # 데이터 기록 날짜보다 이전 date일 경우 skip
        continue
    if usa_date > as_list[-1]: # 데이터 기록 날짜 이후 이면 pass
        break
    replace_date =  usa_replace['replace_date'].iloc[i]
    idx = as_list.index(replace_date)
    as_list[idx] = usa_date

df_usa.index =  as_list # date 일치시킴    
df_usa = df_usa[~df_usa.index.isin(usa_delete['date_usa'])]

In [15]:
df_usa.tail()

,dji_cr,dji_cr_2,dji_f_cr,dji_f_cr_2,dxy_cr,dxy_cr_2,ixic_f_cr,ixic_f_cr_2,ixic_cr,ixic_cr_2,...,ixk_cr,ixk_cr_2,ixtr_cr,ixtr_cr_2,ixut_cr,ixut_cr_2,nbi_cr,nbi_cr_2,bkx_cr,bkx_cr_2
2024-03-14,0.10%,0.70586,0.10,0.73151,-0.16,-0.07777,-0.81,0.61621,-0.54%,0.98943,...,-0.81%,1.44914,-0.02%,0.17420,0.16%,-0.44686,0.26%,-0.19686,0.54%,0.46301
2024-03-15,-0.35%,-0.25593,-0.44,-0.33688,0.56,0.38850,-0.31,-1.11218,-0.30%,-0.84294,...,0.07%,-0.74541,-1.41%,-1.43434,-0.90%,-0.74762,-2.02%,-1.76188,-1.78%,-1.24912
2024-03-18,-0.49%,-0.84150,-0.49,-0.93115,0.07,0.62263,-1.17,-1.47175,-0.96%,-1.26470,...,-1.27%,-1.20060,0.38%,-1.04183,-0.86%,-1.75082,0.21%,-1.80988,0.02%,-1.76334
2024-03-19,0.20%,-0.29618,0.18,-0.31514,0.00,0.06772,0.96,-0.22029,0.82%,-0.15550,...,1.03%,-0.24581,0.07%,0.44877,0.32%,-0.54305,-0.65%,-0.43660,0.49%,0.51005
2024-03-20,0.83%,1.02285,0.81,0.98588,0.42,0.42541,0.21,1.16979,0.39%,1.21216,...,0.40%,1.44197,0.19%,0.25652,-0.11%,0.20698,0.81%,0.15416,0.20%,0.69352


In [16]:
# *****한국 주식 시작 일자, 마지막 일자  지정 ***********
pkl_name= 'kospi.pkl'
df_temp = pd.read_pickle(pkl_directory + pkl_name)

kospi = df_temp['kospi'].iloc[-1]
opening_days_common = list(df_temp['date'])

if kospi == None :
    opening_days_common = list(df_temp['date'][:-1])
    
# 마지막 기록일자 index 찾고 다음 개장일 추가하기 (예측을 원하는 날)
l_date = opening_days_common[-1]
lp1_index = list(OPENING_DAYS_KOR).index(l_date) + 1 # 현재 개장일 이후에 오는 개장일 날짜 index (+1 index)
opening_days_common.append(list(OPENING_DAYS_KOR)[lp1_index])

In [17]:
# 한국 common data df
df_kor = get_common_base(opening_days_common, common_pkl_kor)

In [18]:
df_kor.tail()

,bond_kor_10_cr,bond_kor_10_cr_2,bond_kor_2_cr,bond_kor_2_cr_2,kosdaq_cr,kosdaq_cr_2,kospi_cr,kospi_cr_2,krw_cr,krw_cr_2
date,,,,,,,,,,
2024-03-15,+0.57%,0.56972,+0.69%,0.20852,-0.27,-0.24615,0.94,1.37780,+0.73%,0.89580
2024-03-18,+1.67%,2.24888,+0.83%,1.52649,-0.80,-1.06413,-1.91,-0.99236,+0.54%,1.28179
2024-03-19,+0.73%,0.73314,+1.19%,1.18624,1.59,0.78421,0.71,-1.21085,+0.42%,0.96567
2024-03-20,+1.08%,1.81818,+0.91%,2.10558,-0.29,1.29819,-1.09,-0.38997,+0.08%,0.49676
2024-03-21,-0.37%,0.69869,-0.38%,0.52755,-0.05,-0.33874,1.27,0.16010,+0.16%,0.23873


In [19]:
df_all = df_kor.merge(df_usa, how='left', left_index=True, right_index=True)

In [20]:
df_all.tail()

,bond_kor_10_cr,bond_kor_10_cr_2,bond_kor_2_cr,bond_kor_2_cr_2,kosdaq_cr,kosdaq_cr_2,kospi_cr,kospi_cr_2,krw_cr,krw_cr_2,...,ixk_cr,ixk_cr_2,ixtr_cr,ixtr_cr_2,ixut_cr,ixut_cr_2,nbi_cr,nbi_cr_2,bkx_cr,bkx_cr_2
date,,,,,,,,,,,,,,,,,,,,,
2024-03-15,+0.57%,0.56972,+0.69%,0.20852,-0.27,-0.24615,0.94,1.37780,+0.73%,0.89580,...,0.07%,-0.74541,-1.41%,-1.43434,-0.90%,-0.74762,-2.02%,-1.76188,-1.78%,-1.24912
2024-03-18,+1.67%,2.24888,+0.83%,1.52649,-0.80,-1.06413,-1.91,-0.99236,+0.54%,1.28179,...,-1.27%,-1.20060,0.38%,-1.04183,-0.86%,-1.75082,0.21%,-1.80988,0.02%,-1.76334
2024-03-19,+0.73%,0.73314,+1.19%,1.18624,1.59,0.78421,0.71,-1.21085,+0.42%,0.96567,...,1.03%,-0.24581,0.07%,0.44877,0.32%,-0.54305,-0.65%,-0.43660,0.49%,0.51005
2024-03-20,+1.08%,1.81818,+0.91%,2.10558,-0.29,1.29819,-1.09,-0.38997,+0.08%,0.49676,...,0.40%,1.44197,0.19%,0.25652,-0.11%,0.20698,0.81%,0.15416,0.20%,0.69352
2024-03-21,-0.37%,0.69869,-0.38%,0.52755,-0.05,-0.33874,1.27,0.16010,+0.16%,0.23873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# write common analysis data
globals()['df_all'].to_pickle(directory_for_predict + '0_df_common.pkl')
globals()['df_all'].to_csv(directory_for_predict + '0_df_common.csv')

In [22]:
df_all.tail(10)

,bond_kor_10_cr,bond_kor_10_cr_2,bond_kor_2_cr,bond_kor_2_cr_2,kosdaq_cr,kosdaq_cr_2,kospi_cr,kospi_cr_2,krw_cr,krw_cr_2,...,ixk_cr,ixk_cr_2,ixtr_cr,ixtr_cr_2,ixut_cr,ixut_cr_2,nbi_cr,nbi_cr_2,bkx_cr,bkx_cr_2
date,,,,,,,,,,,,,,,,,,,,,
2024-03-08,+0.53%,-0.23481,0.00%,-0.61029,-0.84,-0.34627,0.23,-0.06719,-0.45%,-0.92182,...,2.11%,2.90996,0.31%,0.95099,-0.81%,-0.45798,0.00%,0.29253,0.27%,-0.39205
2024-03-11,-1.91%,-1.39012,-2.51%,-2.51462,1.14,0.28828,1.24,1.47114,-0.54%,-0.98041,...,-1.66%,0.41928,-0.79%,-0.48138,0.88%,0.06269,0.38%,0.38779,0.14%,0.41490
2024-03-12,+0.57%,0.57212,+0.33%,0.32993,0.31,1.45476,-0.77,0.46155,-0.40%,-0.93342,...,-0.50%,-2.14916,-0.22%,-1.01133,1.50%,2.39395,-0.12%,0.26333,0.12%,0.26239
2024-03-13,-0.15%,0.42156,+0.36%,0.68986,1.57,1.89308,0.83,0.05447,+0.07%,-0.32827,...,2.28%,1.76669,0.19%,-0.02753,-0.60%,0.89212,-0.46%,-0.57888,-0.08%,0.04030
2024-03-14,0.00%,-0.14970,-0.48%,-0.11958,0.02,1.59830,0.44,1.26812,+0.16%,0.23269,...,-0.81%,1.44914,-0.02%,0.17420,0.16%,-0.44686,0.26%,-0.19686,0.54%,0.46301
2024-03-15,+0.57%,0.56972,+0.69%,0.20852,-0.27,-0.24615,0.94,1.37780,+0.73%,0.89580,...,0.07%,-0.74541,-1.41%,-1.43434,-0.90%,-0.74762,-2.02%,-1.76188,-1.78%,-1.24912
2024-03-18,+1.67%,2.24888,+0.83%,1.52649,-0.80,-1.06413,-1.91,-0.99236,+0.54%,1.28179,...,-1.27%,-1.20060,0.38%,-1.04183,-0.86%,-1.75082,0.21%,-1.80988,0.02%,-1.76334
2024-03-19,+0.73%,0.73314,+1.19%,1.18624,1.59,0.78421,0.71,-1.21085,+0.42%,0.96567,...,1.03%,-0.24581,0.07%,0.44877,0.32%,-0.54305,-0.65%,-0.43660,0.49%,0.51005
2024-03-20,+1.08%,1.81818,+0.91%,2.10558,-0.29,1.29819,-1.09,-0.38997,+0.08%,0.49676,...,0.40%,1.44197,0.19%,0.25652,-0.11%,0.20698,0.81%,0.15416,0.20%,0.69352
